### Imports

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pickle

### Carregar o dataset

In [6]:
df = pd.read_csv('../files/result.csv')

In [7]:
df

,title,text,date,category,subcategory,title_word_count,text_word_count
0,"Haverá pressão por transparência no 'brexit', ...","As negociações do ""brexit"" devem ocorrer a por...",2017-03-31,mundo,NaN,10,197
1,Empresário confirma ter pago propina para obte...,"O empresário Augusto Ribeiro de Mendonça Neto,...",2015-03-02,poder,NaN,10,898
2,Falta de ataques do São Paulo surpreende corin...,Cássio sorriu e pareceu sem jeito com a pergun...,2015-02-19,esporte,NaN,10,261
3,Líderes de comissão do Senado não veem sinais ...,Os líderes dos partidos Republicano e Democrat...,2017-03-16,mundo,NaN,12,398
4,Renan opera contra governo e reforma trabalhis...,Um acordo que contou com a articulação de Rena...,2017-03-05,mercado,NaN,11,559
...,...,...,...,...,...,...,...
51491,"Potencial da saúde ainda é pouco aproveitado, ...",Start-ups da área de saúde nos EUA levantaram ...,2015-07-13,mercado,NaN,9,663
51492,Vítima de racismo nega convite do Chelsea para...,Após ser vítima de atos racistas de torcedores...,2015-02-21,esporte,NaN,11,175
51493,Coreia do Sul promete resgatar balsa um ano ap...,"Um ano após o naufrágio da balsa Sewol, no qua...",2015-04-16,mundo,NaN,10,526
51494,Temer convoca PMDB para evento que debaterá te...,O vice-presidente Michel Temer (PMDB) aparece ...,2015-11-11,poder,NaN,11,280


### Baixar Recursos do NLTK

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/roger/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/roger/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/roger/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Definir Stop Words

In [9]:
stop_words = set(stopwords.words('portuguese'))


### Criar um Lematizador

In [10]:
lemmatizer = WordNetLemmatizer()

### Função de pré-processamento de texto

In [11]:
def preprocess_text(text):
    # Tokenização
    tokens = word_tokenize(text.lower())
    # Remoção de stopwords e lematização
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

### Aplicar pré-processamento ao texto

In [12]:
df['processed_text'] = df['text'].apply(preprocess_text)

Verificando a coluna criada com o texto processado

In [13]:
df.head()

,title,text,date,category,subcategory,title_word_count,text_word_count,processed_text
0,"Haverá pressão por transparência no 'brexit', ...","As negociações do ""brexit"" devem ocorrer a por...",2017-03-31,mundo,NaN,10,197,negociações brexit devem ocorrer porta fechada...
1,Empresário confirma ter pago propina para obte...,"O empresário Augusto Ribeiro de Mendonça Neto,...",2015-03-02,poder,NaN,10,898,empresário augusto ribeiro mendonça neto sócio...
2,Falta de ataques do São Paulo surpreende corin...,Cássio sorriu e pareceu sem jeito com a pergun...,2015-02-19,esporte,NaN,10,261,cássio sorriu pareceu jeito pergunta tentou di...
3,Líderes de comissão do Senado não veem sinais ...,Os líderes dos partidos Republicano e Democrat...,2017-03-16,mundo,NaN,12,398,líderes partidos republicano democrata comissã...
4,Renan opera contra governo e reforma trabalhis...,Um acordo que contou com a articulação de Rena...,2017-03-05,mercado,NaN,11,559,acordo contou articulação renan calheiros líde...


### Dividir o dataset em conjunto de treinamento e teste

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['category'], test_size=0.2, random_state=42)

### Vetorização TF-IDF

In [15]:
# 3000 palavras mais importantes
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

### Treinar o classificador Naive Bayes

In [16]:
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

MultinomialNB()

### Prever as categorias das notícias de teste

In [17]:
y_pred = nb_classifier.predict(X_test_tfidf)

### Avaliar o desempenho do classificador

In [18]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     ciencia       0.81      0.58      0.68       138
      comida       0.90      0.47      0.62        79
   cotidiano       0.79      0.87      0.83      1702
    educacao       0.81      0.86      0.83       208
     esporte       0.99      0.93      0.95      1945
     mercado       0.87      0.84      0.86      2124
       mundo       0.87      0.90      0.89      1732
       poder       0.85      0.89      0.87      2177
     turismo       0.84      0.68      0.75       195

    accuracy                           0.87     10300
   macro avg       0.86      0.78      0.81     10300
weighted avg       0.87      0.87      0.87     10300



### Salvando o vetorizador TF-IDF

In [20]:
with open('../data/tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)


### Salvando o classificador Naive Bayes

In [21]:
with open('../data/nb_classifier.pkl', 'wb') as f:
    pickle.dump(nb_classifier, f)

In [22]:
df['text'].iloc[0]


'As negociações do "brexit" devem ocorrer a portas fechadas, mas haverá pressão por transparência, diz à Folha a irlandesa Emily O\'Reilly, a ombudsman da União Europeia.  Sua equipe será responsável por pressionar negociadores europeus a divulgar seus planos e prestar contas. "As pessoas não querem acordar daqui a dois anos e descobrir, como fato consumado, o que vai acontecer", diz. "Querem ter uma ideia durante o processo. O que está acontecendo é enorme."  O\'Reilly ocupa desde 2013 o cargo de ombudsman, de alto escalão dentro da burocracia europeia. Baseada em Estrasburgo, no leste francês, recebe denúncias de cidadãos contra instituições do bloco.  O "brexit", diz ela, será uma de suas preocupações. "Negociadores não poderão ir a uma sala e discutir de maneira imune. As regras mudaram com as redes sociais."  Movimentos populistas, como o que decidiu o "brexit", culpam o bloco pelas mazelas de seus países. É uma perspectiva que pode influenciar também as eleições francesas, em abr